In [23]:
import json
from utils.vocabulary import *
bias_type = "race"
model = "llama_3b"
methods = ["Attention", "Saliency", "DeepLift", "InputXGradient", "IntegratedGradients", "Occlusion", "KernelShap"]
data = "jigsaw"
if data == "civil":
    num_examples = {"race": 2000, "gender": 2000, "religion": 1000}
elif data == "jigsaw":
    num_examples = {"race": 400, "gender": 800, "religion": 200}
expl_predictions = {}
fairness_predictions = {}
fairness_file = f"/scratch/yifwang/new_fairness_x_explainability/decoder_results_{data}/{model}_{data}_{bias_type}_test_{num_examples[bias_type]}/zero_shot/fairness/fairness_{bias_type}_test_individual_stats.json"
with open(fairness_file, "r") as f:
    fairness_predictions = json.load(f)
for group in SOCIAL_GROUPS[bias_type]:
    fairness_predictions[group] = fairness_predictions["Predictions"][group][group]

for method in methods: 
    
    for group in SOCIAL_GROUPS[bias_type]:
        explanation_path = f"/scratch/yifwang/new_fairness_x_explainability/decoder_results_{data}/{model}_{data}_{bias_type}_test_{num_examples[bias_type]}/zero_shot/explanations/{method}_{group}_test_explanations.json"
        with open(explanation_path, "r") as f:
            explanations = json.load(f)
        for aggregation in explanations.keys():
            if aggregation not in expl_predictions:
                expl_predictions[aggregation] = {}
            expl_predictions[aggregation][group] = [expl[0]["predicted_class"] for expl in explanations[aggregation]]

            

In [24]:
# see whether the explanations are consistent with the fairness predictions
for aggr in expl_predictions.keys():
    #print(aggr)
    for group in SOCIAL_GROUPS[bias_type]:
        #print(group, len(expl_predictions[aggr][group]), len(fairness_predictions[group]))
        expl_preds = expl_predictions[aggr][group]
        fairness_preds = fairness_predictions[group]

        match = [1 if expl_preds[i] == fairness_preds[i] else 0 for i in range(len(expl_preds))]
        #print(len(match) - sum(match), sum(match), len(match))
        # find ids of mismatches
        mismatches = [i for i in range(len(match)) if match[i] == 0]
        #print("Mismatches:", mismatches)
        if len(mismatches) > 0:
            print(group, aggr, "Mismatches:", len(mismatches), "out of", len(match))
            print("Mismatch indices:", mismatches)


black IntegratedGradients_L2 Mismatches: 16 out of 400
Mismatch indices: [9, 22, 63, 114, 129, 192, 197, 209, 217, 265, 270, 278, 342, 352, 362, 391]
white IntegratedGradients_L2 Mismatches: 13 out of 400
Mismatch indices: [19, 23, 55, 71, 121, 126, 167, 185, 232, 236, 351, 379, 395]
black IntegratedGradients_mean Mismatches: 16 out of 400
Mismatch indices: [9, 22, 63, 114, 129, 192, 197, 209, 217, 265, 270, 278, 342, 352, 362, 391]
white IntegratedGradients_mean Mismatches: 13 out of 400
Mismatch indices: [19, 23, 55, 71, 121, 126, 167, 185, 232, 236, 351, 379, 395]
